## Set up

In [ ]:
import sys
sys.path.append('./scripts/')
import os

import matplotlib.pyplot as plt
import seaborn as sns
import math
import copy
import numpy as np
sns.set_style("darkgrid")
from PIL import Image
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

import imp
import my_datasets
import utilities 
imp.reload(my_datasets) 
imp.reload(utilities) 

In [ ]:
dataset='ilsvrc12fine'
paths, count, y, idx_to_labels =  my_datasets.get_dataset(dataset)

print(count, len(paths))

In [ ]:
# For ilsvrc12fine dataset, paths are mapped differently
if dataset=='ilsvrc12fine':
    idxs=np.arange(0, 1281167, 10) 
    classes=np.unique(y[idxs])
    ppaths=[paths[i] for i in idxs]
    paths=ppaths

In [ ]:
layer='Mixed_7b.cat_2'
SAVEFOLD0=f'../outputs/{dataset}'

SAVEFOLD=f"{SAVEFOLD0}/{layer}/"

In [ ]:
# gradients_wrt_conv_layer=np.load(f"{SAVEFOLD}/gradients_wrt_conv_layer.npy")
predictions=np.load(f"{SAVEFOLD}/predictions.npy")
conv_maps=np.load(f"{SAVEFOLD}/conv_maps.npy")

# pvh=np.load(f"{SAVEFOLD}/eigenvectors.npy",allow_pickle=True)

In [ ]:
# conv_maps.shape # (10000, 2048, 8, 8)
# GAP since featre maps of size 8*8 for layer Mixed_7b.cat_2
conv_maps_avg = conv_maps.mean(3).mean(2)
# conv_maps_avg.shape # (10000, 2048)

In [ ]:
projections = np.zeros(conv_maps_avg.shape) # .shape (10000, 2048)
for evec in range(len(pvh)):
    projections[:, evec] = np.dot(conv_maps_avg, pvh[evec,:].T)

In [ ]:
transforms = "None" # "SVD" / None / "standardise" / "normalise"

In [ ]:
scale = StandardScaler()
normalise = MinMaxScaler()

standardised_data = scale.fit_transform(conv_maps_avg) 
normalised_data = normalise.fit_transform(conv_maps_avg) # .shape (10000, 2048)

In [ ]:
if transforms == "standardise":
    activations = standardised_data
    print("Standardise")
elif transforms == "normalise": 
    activations = normalised_data
    print("Normalised")
elif transforms == "SVD": 
    activations = projections
    print("SVD")
else: 
    activations = conv_maps_avg
    print("Raw activations")

## Boxplots for neuron activations on Imagenet

In [ ]:
if not os.path.exists(SAVEFOLD+"boxplots"):
    os.mkdir(SAVEFOLD+"boxplots")

In [ ]:
num_neurons = conv_maps.shape[1]

for hundreds in range(math.ceil(num_neurons/100)):
    boxplot_f = f"{SAVEFOLD}/activation_boxplots/hundreds_{hundreds}.png"

    if not os.path.exists(boxplot_f):
        fig, ax = plt.subplots(10, 1, figsize=(8, 8 * 10)) # 
        
        for start in range(hundreds*100,min((hundreds+1)*100,num_neurons),10):
            ax[(start//10)-(hundreds*10)].set_title(f"Distribution of neuron {start}-{start + 9} activations")
            ax[(start//10)-(hundreds*10)].set_ylim([0, 3]) 
            sns.boxplot(activations[:,start:start+10], ax = ax[(start//10)-(hundreds*10)])
        
        fig.savefig(boxplot_f, bbox_inches="tight") 
        print("saved file!")
    else: 
        print("File already exists!")

## Random  analysis

In [ ]:
if not os.path.exists(SAVEFOLD+"max_activating_ims"):
    os.mkdir(SAVEFOLD+"max_activating_ims")

In [ ]:
num_neurons = conv_maps.shape[1]
plt.clf()
sns.set()

for hundreds in range(math.ceil(num_neurons/100)):
    max_activating_ims_f = f"{SAVEFOLD}/max_activating_ims/hundreds_{hundreds}.png"
    neuron_range = range(hundreds*100,min(hundreds*100+100,num_neurons))
    if not os.path.exists(max_activating_ims_f):
        fig, ax = plt.subplots(100, 11, figsize=(22, 200)) # (10, 11, figsize=(22, 20))
        ax = ax.flatten()
        i=0
        for neuron in neuron_range: ### change
            if neuron > num_neurons:
                break
            top_10 = activations[:,neuron].argsort()[-10:][::-1]
            ax[i].text(1.0, 0.5, "Neuron "+str(neuron), ha='right', va='center', family='sans-serif', size=16)
            ax[i].axis('off')
            i+=1 
            for act in top_10:
                # print("Image: " + str(act))
                im = Image.open(paths[act])
                # im.show() 
                ax[i].imshow(im)
                ax[i].axis('off')
                ax[i].set_title(str(act)+" : "+str(round(activations[act,neuron],3)), size=12)
                i+=1
        plt.show()

        fig.savefig(max_activating_ims_f, bbox_inches="tight") 
        print("saved file!")
    else: 
        print("File already exists!")

In [ ]:
neuron # 240
neuron_range # range(140, 1500)

### SVD directions

In [ ]:
# if transforms == "SVD": 
#     if not os.path.exists(SAVEFOLD+"evec_max_activating_ims"):
#         os.mkdir(SAVEFOLD+"evec_max_activating_ims")

In [ ]:
# if transforms == "SVD": 
#     num_neurons = conv_maps.shape[1]
#     plt.clf()
#     sns.set()

#     for hundreds in range(math.ceil(num_neurons/100)):
#         max_activating_ims_f = f"{SAVEFOLD}/evec_max_activating_ims/hundreds_{hundreds}.png"
#         neuron_range = range(hundreds*100,min(hundreds*100+100,num_neurons))
#         if not os.path.exists(max_activating_ims_f):
#             fig, ax = plt.subplots(100, 11, figsize=(22, 200)) # (10, 11, figsize=(22, 20))
#             ax = ax.flatten()
#             i=0
#             for neuron in neuron_range: ### change
#                 if neuron > num_neurons:
#                     break
#                 top_10 = activations[:,neuron].argsort()[-10:][::-1]
#                 ax[i].text(1.0, 0.5, "Direction "+str(neuron), ha='right', va='center', family='sans-serif', size=16)
#                 ax[i].axis('off')
#                 i+=1 
#                 for act in top_10:
#                     # print("Image: " + str(act))
#                     im = Image.open(paths[act])
#                     # im.show() 
#                     ax[i].imshow(im)
#                     ax[i].axis('off')
#                     ax[i].set_title(str(act)+" : "+str(round(activations[act,neuron],3)), size=12)
#                     i+=1
#             plt.show()

#             fig.savefig(max_activating_ims_f, bbox_inches="tight") 
#             print("saved file!")
#         else: 
#             print("File already exists!")

## Maximally activating dataset examples

### activations for one image

In [ ]:
print(idx_to_labels[f'{int(y[0])}'][1])
Image.open(paths[0])

In [ ]:
# np.argmax(activations[0]) # 1215
# activations[0].argsort()[-5:][::-1] # array([1215, 1527, 1620, 1876, 1624])

In [ ]:
# example 0 activations
print(activations[0])
plt.plot(activations[0])